# TAL aplicado al análisis del discurso de los medios de prensa 📰🤓🔥


El proyecto consiste en entrenar y evaluar varios modelos de clasificación supervisada capaz de clasificar una noticia según la taxonomía siguiente: 


- mundo = 0 
- economía = 1 
- política y conflictos = 2 
- ciencia y tecnología = 3
- catástrofes y accidentes = 4 
- cultura y arte = 5 
- deporte = 6 
- ecología y planeta = 7
- crimen, delitos y justicia = 8 
- salud = 9

- Hito Unidad 1 (29 de septiembre): Datasets de entrenamiento y test + primer modelo baseline

- Hito Unidad 2 (27 de octubre): Implementación y experimentos de varios modelos de clasificación

- Hito Proyecto (15 de diciembre): Evaluación y comparación de los modelos de los distintos equipos + integración de los mejores modelos en la arquitectura Sophia2.


### índex

1. [Importación del dataset](a)
2. [Extracción de tópicos vía RE en URLs](b)
3. [Extracción de términos clave de columnas title+text](b)
4. [Clasificación del dataset de entrenamiento](c)
5. [Balance de clases dataset de entrenamiento](d)
6. [Importación de los dataset restantes](e)
7. [Dividir dataset en entrenamiento y test](f)
8. [Dividir dataset entrenamiento en entrenamiento y validación](g)

## Generando dataset de entrenamiento

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import re
import multiprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# NLP
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec, CoherenceModel
#import pyLDAvis
#import pyLDAvis.gensim_models  # don't skip this
import nltk 
#nltk.download('stopwords') 
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_sm")
print("versión de Spacy: ", spacy.__version__)

versión de Spacy:  3.1.3


### 1. Importación de dataset <a name="a"></a>

In [18]:
frames = [pd.read_csv("data/chile_2020-09.csv"), pd.read_csv("data/chile_2020-10.csv"),pd.read_csv("data/chile_2020-11.csv"), pd.read_csv("data/chile_2020-12.csv"), pd.read_csv("data/chile_2021-01.csv"), pd.read_csv("data/chile_2021-02.csv"), pd.read_csv("data/chile_2021-03.csv"), pd.read_csv("data/chile_2021-04.csv")]
dataset = pd.concat(frames)
dataset['content'] = dataset['title'] + " " + dataset['text'] 

print("Largo de dataset: ", len(dataset))
dataset= dataset[dataset['text'].notna()]
        
dataset.drop(columns=['country','year','date','id', 'id_journalist','title'], inplace=True)

dataset['value'] = ""

Largo de dataset:  69702


In [19]:
dataset

,media_outlet,url,text,content,value
0,horas24,https://www.24horas.cl/coronavirus/tia-pikachu...,"Ha sufrido los embates del carro lanza aguas,...","""Tía Pikachu"" por parte empadronado: ""Estoy co...",
3,horas24,https://www.24horas.cl/nacional/rm-alcanza-sus...,"La ministra del Medio Ambiente, Carolina Schm...",RM alcanza sus mejores índices de calidad de a...,
6,horas24,https://www.24horas.cl/nacional/paro-de-camion...,"Esta jornada, la sexta de paralización de los...",Paro de camioneros: Supermercados preocupados ...,
7,horas24,https://www.24horas.cl/economia/sernac-recibio...,"Luego de la primera jornada de CyberDay, el S...",Sernac recibió más de 400 reclamos durante la ...,
8,horas24,https://www.24horas.cl/nacional/denuncian-desa...,Habitantes de diferentes comunas del sur del ...,Denuncian desabastecimiento en el sur del país...,
...,...,...,...,...,...
1988,emol,https://www.emol.com/noticias/Deportes/2021/04...,Este lunes Nicolás Jarry (507°) debía enfrent...,El duelo entre Jarry y el argentino Ugo Carabe...,
1989,emol,https://www.emol.com/noticias/Deportes/2021/04...,Este lunes se volvió a actualizar el ranking ...,Ranking ATP: Hubo importante cambio en el top ...,
1990,emol,https://www.emol.com/noticias/Deportes/2021/04...,golagol('/especiales/2020/deportes/gol-gol/Cha...,Resultados de las semifinales de ida de la Cha...,
1991,emol,https://www.emol.com/noticias/Deportes/2021/04...,golagol('/especiales/2021/deportes/gol-gol/Cop...,Resultados de los chilenos en la Copa Sudameri...,


### 2.  Extracción de tópicos vía RE en URL <a name="b"></a>

In [27]:
for index, row in dataset.iterrows():
    url=row['url']
    obj = re.findall('(\w+)://([\w\-\.]+)/([\w\-]+).([\w\-]+)', url) 
    topic=obj[0][2]
    
    dataset.loc[index,'topic'] = topic
    
    if topic == 'mundo': 
        row['value'] = 0
    if topic == 'economia': 
        row['value'] = 1
    if topic == 'negocios': 
        row['value'] = 1      
    if topic == 'politica': 
        row['value'] = 2
    if topic == 'ciencia': 
        row['value'] = 3
    if topic == 'tecnologia': 
        row['value'] = 3    
    if topic == 'catastrofes': 
        row['value'] = 4      
    if topic == 'cultura': 
        row['value'] = 5
    if topic == 'artes': 
        row['value'] = 5
    if topic == 'deportes': 
        row['value'] = 6
    if topic == 'el-deportivo': 
        row['value'] = 6
    if topic == 'medio-ambiente': 
        row['value'] = 7
    if topic == 'justicia': 
        row['value'] = 8
    if topic =='reportaje-investigacion': 
        row['value'] = 8    
    if topic == 'salud': 
        row['value'] = 9 

dataset['value'].value_counts()

     56237
0     3911
2     3281
6     2596
8      536
1      328
5      234
7      151
9       65
Name: value, dtype: int64

In [28]:
dataset

,media_outlet,url,text,content,value,topic
0,horas24,https://www.24horas.cl/coronavirus/tia-pikachu...,"Ha sufrido los embates del carro lanza aguas,...","""Tía Pikachu"" por parte empadronado: ""Estoy co...",,noticias
3,horas24,https://www.24horas.cl/nacional/rm-alcanza-sus...,"La ministra del Medio Ambiente, Carolina Schm...",RM alcanza sus mejores índices de calidad de a...,,noticias
6,horas24,https://www.24horas.cl/nacional/paro-de-camion...,"Esta jornada, la sexta de paralización de los...",Paro de camioneros: Supermercados preocupados ...,,noticias
7,horas24,https://www.24horas.cl/economia/sernac-recibio...,"Luego de la primera jornada de CyberDay, el S...",Sernac recibió más de 400 reclamos durante la ...,1,noticias
8,horas24,https://www.24horas.cl/nacional/denuncian-desa...,Habitantes de diferentes comunas del sur del ...,Denuncian desabastecimiento en el sur del país...,,noticias
...,...,...,...,...,...,...
1988,emol,https://www.emol.com/noticias/Deportes/2021/04...,Este lunes Nicolás Jarry (507°) debía enfrent...,El duelo entre Jarry y el argentino Ugo Carabe...,,noticias
1989,emol,https://www.emol.com/noticias/Deportes/2021/04...,Este lunes se volvió a actualizar el ranking ...,Ranking ATP: Hubo importante cambio en el top ...,,noticias
1990,emol,https://www.emol.com/noticias/Deportes/2021/04...,golagol('/especiales/2020/deportes/gol-gol/Cha...,Resultados de las semifinales de ida de la Cha...,,noticias
1991,emol,https://www.emol.com/noticias/Deportes/2021/04...,golagol('/especiales/2021/deportes/gol-gol/Cop...,Resultados de los chilenos en la Copa Sudameri...,,noticias


Tenemos muchas noticias no clasificadas, por lo que es necesario utilizar otro mecanísmo para clasificar las noticias restantes.

### 3.  Extracción de términos clave de columna title+text <a name="b"></a>

In [ ]:
matcher = Matcher(nlp.vocab)

# Pattern 1: NOUN de NOUN
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

# Pattern 2: NOUN ADJ
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

# Pattern 3: 
pattern_3 = [{"POS": "NOUN"}]
matcher.add("NOUN", [pattern_3])



    Bloque de alto coste computacional ⬇️

In [ ]:
dataset['terms'] = ""
for index, row in dataset.iterrows():
    doc = nlp(row['content'].lower())

    matches = matcher(doc)
    categories = ""
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        categories = categories + span.text + "; "
    
    row['terms'] = categories
    
display(dataset.head(3))

In [ ]:
from collections import Counter
dataset["most_common"] = ""
for index, row in dataset.iterrows():
    split_it = row['terms'].split("; ")
    counter = Counter(split_it)
    row['most_common'] = counter.most_common()[0][0]

display(dataset.head(5),dataset['most_common'].value_counts())

### 4. Clasificación del dataset de entrenamiento <a name="c"></a>
Aplicaremos LDA a cada noticia y extraeremos sus tópicos, luego evaluaremos esos tópicos con las categorias deseadas.


In [ ]:
dataset['category'] = ""
#for index, row in dataset.iterrows():
#    for index in topics:

dataset

### 5.  Balance de clases dataset de entrenamiento <a name="d"></a>
Visualizamos la densidad de los tópicos con tal de combatir el sobreajuste si corresponde

In [ ]:
fig, ax = plt.subplots(1, figsize=(9,5), tight_layout=True)
ax.hist(dataset['category'],edgecolor = 'black',bins=dataset['category'].unique())
plt.xticks(rotation='vertical', fontsize=7)
plt.ylabel("densidad")
plt.show()